In [ ]:
# default_exp importers.icloud

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from integrators.data.basic import *
from integrators.indexers.facerecognition.photo import IPhoto, show_images
from integrators.importers.importer import ImporterBase

In [ ]:
from pyicloud import PyiCloudService
import click
from pathlib import Path

# ICloudImporter

In [ ]:
# export
def select_device(api):
    devices = api.trusted_devices
    if devices is None: return None
    if len(devices) > 1:
        for i, device in enumerate(devices):
            auth_str = device.get(f'deviceName', f"SMS to {device.get('phoneNumber')}")
            print(f"  {i}: {auth_str}")

        device = click.prompt('Which device would you like to use?', default=0)
        device = devices[device]
    else:
        device = devices[0]
    return device
    

def handle_2sa(api):
    print("Two-step authentication required. Your trusted devices are:")
    
    device = select_device(api)
    if device is None: 
        print("no devices found, already logged in?")
        return
    print(device)                 
    if not api.send_verification_code(device):
        print("Failed to send verification code, might be due to too many requests")
        assert False
    code = click.prompt('Please enter validation code')
    if not api.validate_verification_code(device, code):
        print("Failed to verify verification code")
    else:
        print("succes")

## Login

In [ ]:
# export
class ICloudImporter(ImporterBase):
    DEFAULT_VERSION = 'original'
    
    def __init__(self, apple_id=None, password=None, *args, **kwargs):
        self.api = PyiCloudService(apple_id, password)
        super().__init__(*args, **kwargs)
    
    def get_images(self, n=None):
        n = 1e9 if n is None else n
        res = []
        
        for i, p in enumerate(self.api.photos.all):
            if i > n: break
            meta = p.versions.get(self.DEFAULT_VERSION, dict())
            fname = meta.get('filename', '')
            if str(Path(fname).suffix) not in [".jpg", ".JPG", ".jpg", "PNG"]: continue
            print(fname)

            photo = self.get_photo(p, meta=meta)
            res.append(photo)
        return res
    
    @classmethod
    def get_photo(self, photo_obj, meta=None):
        meta = dict() if meta is None else meta
        print(meta.get("filename", None))
        res = photo_obj.download(self.DEFAULT_VERSION)
        data = res.raw.read()
        return IPhoto.from_bytes(data)

In [ ]:
def get_credentials(file=MEMRI_DIR / "credentials_icloud.txt", manual=False):
    # In CI, we are storing files in ~/.memri/credentials_icloud.txt
    if file.exists() and not manual:
        apple_id, password = read_file(MEMRI_DIR / "credentials_icloud.txt").splitlines()
    else:
        apple_id = click.prompt('Enter your apple id (this is not stored)')
        password = click.prompt('Enter your password (this is not stored)')
    return apple_id, password

In [ ]:
apple_id, password = get_credentials(manual=True)

In [ ]:
importer = ICloudImporter(apple_id=apple_id, password=password)
handle_2sa(importer.api)

## Run

In [ ]:
photos = importer.get_images(n=10)

In [ ]:
photos

In [ ]:
show_images(photos)

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()